In [1]:
import os
import pandas as pd
import numpy as np
from PIL import Image, ImageFile
from mtcnn import MTCNN
from tqdm import tqdm

In [2]:
# Cho phép load ảnh lỗi nhẹ (truncated)
ImageFile.LOAD_TRUNCATED_IMAGES = True

def process_dataset(input_root, output_root, size=(256, 256), max_images=None):
    """
    Xử lý dataset: crop khuôn mặt, resize, giữ nguyên cấu trúc thư mục
    input_root: thư mục gốc (vd: dataset/)
    output_root: thư mục đầu ra (vd: dataset_processed/)
    size: kích thước ảnh sau resize
    max_images: số lượng ảnh tối đa mỗi folder con để test nhanh (None = full)
    """
    detector = MTCNN()
    os.makedirs(output_root, exist_ok=True)

    # Tạo file log để ghi lỗi
    log_path = os.path.join(output_root, "missing_images.txt")
    with open(log_path, "w", encoding="utf-8") as log:
        log.write("Danh sách ảnh lỗi hoặc bị thiếu:\n")

    # Duyệt qua các folder con (photo, photo2)
    for folder in os.listdir(input_root):
        folder_path = os.path.join(input_root, folder)
        if not os.path.isdir(folder_path):
            continue

        print(f"\n🔹 Đang xử lý thư mục: {folder}")
        input_dir = folder_path
        output_dir = os.path.join(output_root, folder)
        os.makedirs(os.path.join(output_dir, "live"), exist_ok=True)
        os.makedirs(os.path.join(output_dir, "not_live"), exist_ok=True)

        csv_path = os.path.join(input_dir, "list.csv")
        if not os.path.exists(csv_path):
            print(f"⚠️ Không tìm thấy file list.csv trong {input_dir}")
            continue

        df = pd.read_csv(csv_path)

        # Giới hạn số lượng ảnh để test nhanh
        if max_images:
            df = df.sample(min(max_images, len(df)), random_state=42)

        new_rows = []

        for _, row in tqdm(df.iterrows(), total=len(df), desc=f"Processing {folder}"):
            rel_path = row["filename"]
            label = row["label"]
            in_path = os.path.join(input_dir, rel_path)
            out_path = os.path.join(output_dir, rel_path)

            try:
                img = Image.open(in_path).convert("RGB")
                boxes = detector.detect_faces(np.array(img))

                # Nếu phát hiện khuôn mặt → crop, nếu không thì resize toàn ảnh
                if len(boxes) > 0:
                    x, y, w, h = boxes[0]["box"]
                    x, y = max(0, x), max(0, y)
                    face = img.crop((x, y, x + w, y + h)).resize(size)
                else:
                    face = img.resize(size)

                os.makedirs(os.path.dirname(out_path), exist_ok=True)
                face.save(out_path)
                new_rows.append({"filename": rel_path, "label": label})

            except Exception as e:
                print(f"⚠️ Lỗi khi xử lý {in_path}: {e}")
                with open(log_path, "a", encoding="utf-8") as log:
                    log.write(f"{in_path} - {e}\n")
                continue

        # Ghi file CSV mới
        pd.DataFrame(new_rows).to_csv(os.path.join(output_dir, "list.csv"), index=False)
        print(f"✅ Hoàn tất {folder}: {len(new_rows)} ảnh hợp lệ")
        print(f"📄 CSV mới: {os.path.join(output_dir, 'list.csv')}")

    print(f"\n✨ Tất cả hoàn tất! Log lỗi nằm tại: {log_path}")

# 🔧 --- CHẠY THỬ ---
# Nếu chỉ muốn test nhanh: đặt max_images = 50 (hoặc số nhỏ hơn)
process_dataset(
    input_root="dataset",             # thư mục gốc chứa photo + photo2
    output_root="dataset_processed",  # thư mục đầu ra
    size=(256, 256)                   # resize ảnh
)



🔹 Đang xử lý thư mục: photo


Processing photo:   6%|▌         | 258/4468 [02:55<35:21,  1.98it/s]   

⚠️ Lỗi khi xử lý dataset\photo\not_live/1890245e-b1fc-11ed-a57b-b42e9920aa06_face_image_live_183bcc7e-b1fc-11ed-8512-b42e9920aa05.jpg: [Errno 2] No such file or directory: 'D:\\CODE\\face-anti-spoofing\\dataset\\photo\\not_live\\1890245e-b1fc-11ed-a57b-b42e9920aa06_face_image_live_183bcc7e-b1fc-11ed-8512-b42e9920aa05.jpg'


Processing photo:  10%|▉         | 429/4468 [04:40<34:19,  1.96it/s]  

⚠️ Lỗi khi xử lý dataset\photo\live/00ff6296-b13e-11ed-b531-b42e9920aa06_face_image_live_003dca1e-b13e-11ed-8512-b42e9920aa05.jpg: [Errno 2] No such file or directory: 'D:\\CODE\\face-anti-spoofing\\dataset\\photo\\live\\00ff6296-b13e-11ed-b531-b42e9920aa06_face_image_live_003dca1e-b13e-11ed-8512-b42e9920aa05.jpg'


...

⚠️ Lỗi khi xử lý dataset\photo2\live/b6647b08-b1e8-11ed-b531-b42e9920aa06_face_image_live_b5d7569c-b1e8-11ed-80d1-b42e9920aa05.jpg: [Errno 2] No such file or directory: 'D:\\CODE\\face-anti-spoofing\\dataset\\photo2\\live\\b6647b08-b1e8-11ed-b531-b42e9920aa06_face_image_live_b5d7569c-b1e8-11ed-80d1-b42e9920aa05.jpg'


Processing photo2:  93%|█████████▎| 2236/2413 [22:11<01:56,  1.52it/s]

⚠️ Lỗi khi xử lý dataset\photo2\live/7770e0f2-b31a-11ed-a23e-b42e9920aa06_face_image_live_76e57350-b31a-11ed-afb3-b42e9920aa05.jpg: [Errno 2] No such file or directory: 'D:\\CODE\\face-anti-spoofing\\dataset\\photo2\\live\\7770e0f2-b31a-11ed-a23e-b42e9920aa06_face_image_live_76e57350-b31a-11ed-afb3-b42e9920aa05.jpg'


Processing photo2: 100%|██████████| 2413/2413 [23:48<00:00,  1.69it/s]

✅ Hoàn tất photo2: 2399 ảnh hợp lệ
📄 CSV mới: dataset_processed\photo2\list.csv

✨ Tất cả hoàn tất! Log lỗi nằm tại: dataset_processed\missing_images.txt
